In [ ]:
# pandas is a python data analysis library used for dataframes in this script 
# https://pandas.pydata.org/

# psycopg2 is a postgres database adapter for python used for database connection & migration in this script
# http://initd.org/psycopg/docs/

# os is a directory of miscellaneous operating system interfaces for python
# used to decode file names and list all files in a directory in this script
# https://docs.python.org/3/library/os.html

# import json to work with json files

import pandas as pd
import psycopg2 as p
import os
import json

In [ ]:
# This script handles a folder containing a years worth of json play-by-play data.
# This script handles the FLATTENED json files.

# The script:  
# 1) iterates through each week file in the year file
# 2) finds the flattened file
# 3) reads the data into a dataframe 
# 4) normalizes the data into game and play 
# 5) writes each dataframe to a csv
# 6) uploads the csvs to a database's relating tables 


# path to folder with year of files
year = r'C:\Users\mmgri\Desktop\2005'

# creating database connection string
conn = p.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=mmgtest user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

#declare array for dataframes
dfs = []

#iterate thru each week
for week in os.listdir(year):
    
        #get name and path
        weekName = os.fsdecode(week)
        weekPath = os.path.join(year, weekName)
        
        #iterate thru files in week
        for flat in os.listdir(weekPath):
            
            flatName = os.fsdecode(flat)
            flatPath = os.path.join(weekPath, flatName)

            #finding flattened folder
            string = 'flattened'
            
            if flatName == string:

                for game in os.listdir(flatPath):

                    gameName = os.fsdecode(game)
                    gamePath = os.path.join(flatPath, gameName)
                        
                    print(gameName)

                    # find all json files
                    if gameName.endswith(".json"): 

                        # read json file to dataframe
                        df = pd.read_json(os.path.join(gamePath), orient='columns')

                        # add dataframe to list of dataframes
                        dfs.append(df)
                        continue
                    else:
                        continue

# combine list into one dataframe
df = pd.concat(dfs)

# remove commas from descriptions to avoid erros
df['description'] = df['description'].str.replace('[^\w\s]', '')

# normalize dataframes 
dfGame = df[['awayAbbr', 'awayId', 'awayTeam', 'gameId', 'homeAbbr', 'homeId', 'homeTeam', 'week', 'year']]
dfDrive = df[['defenseAbbr', 'defenseId', 'defenseTeam', 'driveIndex', 'gameId', 'offenseAbbr', 'offenseId',
             'offenseTeam', 'quarter']]
dfPlay = df[['awayScore', 'clock', 'defenseId', 'description', 'distance', 'down', 
             'driveIndex', 'endYardLine', 'homeScore', 'offenseId', 
              'playIndex', 'type', 'yardLine', 'yardsGained']]

# remove duplicates from game and drive 
dfGame.drop_duplicates(subset ="gameId", keep = 'first', inplace = True) 
dfDrive.drop_duplicates(subset = ["defenseId", "driveIndex", "offenseId"], keep = 'first', inplace = True)

# array for file names and table names 
fileNames = ['game', 'drive', 'play']

# array for normalized dataframes
dfNormalized = [dfGame, dfDrive, dfPlay]

# loop to add files to postgres database
for df, file in zip(dfNormalized, fileNames):
    print(df.shape)
    print(file)
    
    # writes dataframe to a csv 
    df.to_csv(r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv', sep=',', index=False)
    fileString = r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv'
    
    # loads csv to database 
    with open(fileString, 'r') as f:
        next(f)
        cur = conn.cursor()
        cur.copy_from(f, file, sep=',')
        conn.commit()
     
